# TransformChain

### Imports

In [1]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import chain

### Loading local environment parameters

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### Setting up LLM model

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                   max_tokens=2000,
                   temperature=0.3)

In [4]:
yelp_review = open('yelp_review.txt').read()

In [5]:
print(yelp_review)

TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!

REVIEW:
OH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!

FIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.

NOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!

THEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!

THE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND EXTREMELY KN

### Writing Custom Transformation Functions

Take in the input dictionary text and output after any transformations you want!

In [6]:
def transformer_fun(inputs: dict) -> dict:
    '''
    Notice how this always takes an inputs dictionary.
    Also outputs a dictionary. You can call the output and input keys whatever you want, 
    just make sure to reference it correct in the chain call.
    '''
    # GRAB INCOMING CHAIN TEXT
    text = inputs['text']
    only_review_text = text.split('REVIEW:')[-1]
    lower_case_text = only_review_text.lower()
    return {'output':lower_case_text}

## Old syntax

In [7]:
transform_chain = TransformChain(input_variables=['text'],
                                 output_variables=['output'],
                                 transform=transformer_fun)

In [8]:
template = "Create a one sentence summary of this review:\n{output}"

In [9]:
prompt = ChatPromptTemplate.from_template(template)

summary_chain = LLMChain(llm=llm,
                     prompt=prompt,
                     output_key="review_summary")

C:\Users\JimYi\AppData\Local\Temp\ipykernel_37956\3101234180.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=llm,


In [10]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain, summary_chain],
                                        verbose=True)

In [11]:
result = sequential_chain.invoke(yelp_review)



> Entering new SimpleSequentialChain chain...

oh my goodness, where do i begin? this restaurant is absolutely phenomenal! i went there last night with my friends, and we were blown away by the experience!

first of all, the ambiance is out of this world! the moment you step inside, you're greeted with a warm and inviting atmosphere. the decor is stunning, and it immediately sets the tone for an unforgettable dining experience.

now, let's talk about the food! wow, just wow! the menu is a paradise for food lovers. every dish we ordered was a masterpiece. the flavors were bold, vibrant, and exploded in our mouths. from starters to desserts, every bite was pure bliss!

their seafood platter is a must-try! the freshness of the seafood is unmatched, and the presentation is simply stunning. i have never tasted such delicious and perfectly cooked seafood in my life. it's a seafood lover's dream come true!

the service was exemplary. the staff was attentive, friendly, and extremely knowledg

In [12]:
print(result['output'])

This restaurant offers an unforgettable dining experience with stunning ambiance, exceptional food—especially the seafood platter and chocolate lava cake—and exemplary service, making it a must-visit culinary gem.


## New syntax

In [13]:
transform_chain2 = RunnableLambda(transformer_fun)
template = "Create a one sentence summary of this review:\n{output}"
prompt = ChatPromptTemplate.from_template(template)
summary_chain2 = prompt | llm | StrOutputParser()
sequential_chain2 = (transform_chain2 | summary_chain2).with_config({"verbose": True})
result = sequential_chain2.invoke({"text": yelp_review})
print(result)

This restaurant offers an extraordinary dining experience with stunning ambiance, exceptional food—especially the seafood platter and chocolate lava cake—and exemplary service, making it a must-visit culinary gem.


## Extension

In [14]:
def transformer_fun2(inputs: dict) -> dict:
    '''
    Notice how this always takes an inputs dictionary.
    Also outputs a dictionary. You can call the output and input keys whatever you want, 
    just make sure to reference it correct in the chain call.
    '''
    # GRAB INCOMING CHAIN TEXT
    text = inputs['text']
    upper_case_text = text.title()
    return {'output':upper_case_text}

In [15]:
transform_chain2 = TransformChain(input_variables=['text'],
                                 output_variables=['output'],
                                 transform=transformer_fun2)

In [16]:
summary_chain2 = transform_chain | prompt | llm | StrOutputParser() | transform_chain2

In [17]:
result = summary_chain2.invoke(yelp_review)

In [18]:
print(result['output'])

This Restaurant Offers An Extraordinary Dining Experience With Stunning Ambiance, Exceptional Seafood, Delectable Desserts, And Exemplary Service, Making It A Must-Visit Culinary Gem.


In [19]:
template2 = "Rewrite the following with a title in all cap and the text in the proper capitalization:\n{output}"
prompt2 = ChatPromptTemplate.from_template(template2)

In [20]:
summary_chain3 = transform_chain | prompt | llm | StrOutputParser() | transform_chain2 | prompt2 | llm | StrOutputParser()

In [21]:
result2 = summary_chain3.invoke(yelp_review)

In [22]:
print(result2)

UNFORGETTABLE DINING EXPERIENCE

This restaurant offers an unforgettable dining experience with stunning ambiance, exquisite food, exceptional service, and a must-try seafood platter, making it a hidden gem that every food lover should visit.
